In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt

In [ ]:
# EDF Raw Data Visualizer | Version 1

# Inputs
## edf_1 = path to the first .edf EEG file
## edf_2 = path to a second .edf EEG file
## edf_3 = path to a third .edf EEG file
## n_axes = num of total axes. Default = 3

edf_1 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R03.edf'
edf_2 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R07.edf'
edf_3 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R11.edf'
n_axes = 3


def plot_edf_segment(path, tmin=0, tmax=10, max_channels=8, decimate=4, ax=None):
    raw = mne.io.read_raw_edf(path, preload=False, verbose='ERROR')
    picks = list(range(min(max_channels, raw.info['nchan'])))
    sfreq = raw.info['sfreq']
    start = max(0, int(tmin * sfreq))
    stop = min(raw.n_times, int(tmax * sfreq))
    data = raw.get_data(picks=picks, start=start, stop=stop)
    if decimate > 1:
        data = data[:, ::decimate]
        sfreq = sfreq / decimate
    times = np.arange(data.shape[1]) / sfreq + tmin

    global_max = 0
    for p in [edf_1, edf_2, edf_3]:
        raw = mne.io.read_raw_edf(p, preload=False, verbose='ERROR')
        d = raw.get_data(start=0, stop=int(raw.info['sfreq']*10))
        global_max = max(global_max, np.nanmax(abs(d)))

    print(global_max)

    offsets = np.arange(data.shape[0]) * global_max * 1.2
    if ax is None:
        ax = plt.gca()
    for idx, trace in enumerate(data):
        ax.plot(times, trace + offsets[idx], linewidth=0.6)
    ax.set_yticks(offsets)
    ax.set_yticklabels([raw.ch_names[p] for p in picks], fontsize=7)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Region')

    title = str(path)
    ax.set_title(title[-6:-4])


fig, axes = plt.subplots(1, n_axes, figsize=(12, 6), sharey=True)

plot_edf_segment(edf_1, ax=axes[0])
plot_edf_segment(edf_2, ax=axes[1])
plot_edf_segment(edf_3, ax=axes[2])

fig.tight_layout()
plt.show()


In [ ]:
# EDF Raw Data Visualizer | Version 2

# Inputs
## edf_1 = path to the first .edf EEG file
## edf_2 = path to a second .edf EEG file
## edf_3 = path to a third .edf EEG file
## n_axes = num of total axes. Default = 3

edf_1 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R03.edf'
edf_2 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R07.edf'
edf_3 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R11.edf'
n_axes = 3


def plot_edf_segment(path, ax, seconds=10, max_channels=8):
    raw = mne.io.read_raw_edf(path, preload=False, verbose='ERROR')
    picks = np.arange(min(max_channels, raw.info['nchan']))
    stop = int(raw.info['sfreq'] * seconds)
    data = raw.get_data(picks=picks, start=0, stop=stop)
    times = np.arange(data.shape[1]) / raw.info['sfreq']

    scale = np.nanmax(np.abs(data))
    offsets = np.arange(data.shape[0]) * 0.000584 * 1.2
    for idx, trace in enumerate(data):
        ax.plot(times, trace + offsets[idx], linewidth=0.6)

    ax.set(
        yticks=offsets,
        yticklabels=[raw.ch_names[p] for p in picks],
        xlabel='Time (s)',
        ylabel='Region',
        title=str(path)[-6:-4],
    )


fig, axes = plt.subplots(1, n_axes, figsize=(12, 6), sharey=True)

plot_edf_segment(edf_1, ax=axes[0])
plot_edf_segment(edf_2, ax=axes[1])
plot_edf_segment(edf_3, ax=axes[2])

fig.tight_layout()
plt.show()


In [ ]:
task1_subject_matrix = []
events_general, event_id_general = (0, 0)
event_map = {"left": 2, "right": 3} 

for subjectNum in range(1, 2):
    raw_task1_1 = mne.io.read_raw_edf(f"../data/physionet.org/files/eegmmidb/1.0.0/S00{subjectNum}/S001R03.edf", preload=True)
    raw_task1_2 = mne.io.read_raw_edf(f"../data/physionet.org/files/eegmmidb/1.0.0/S00{subjectNum}/S001R07.edf", preload=True)
    raw_task1_3 = mne.io.read_raw_edf(f"../data/physionet.org/files/eegmmidb/1.0.0/S00{subjectNum}/S001R11.edf", preload=True)

    events_general, event_id_general = mne.events_from_annotations(raw_task1_1)

    subjectEntry = [raw_task1_1, raw_task1_2, raw_task1_3]

    task1_subject_matrix.append(subjectEntry)

In [ ]:
all_X = []
all_Y = []

for subject_entry in task1_subject_matrix:
    for raw in subject_entry:
        events, _ = mne.events_from_annotations(raw)
        epochs = mne.Epochs(
            raw,
            events,
            event_id=event_map,
            tmin=0.0,
            tmax=4.0,
            baseline=None,
            preload=True
        )

        data = epochs.get_data()  # (M, C, T)
        X = np.transpose(data, (1, 2, 0))  # (C, T, M)

        labels = epochs.events[:, -1]
        Y = np.where(labels == 2, -1, 1).reshape(-1, 1)

        all_X.append(X)
        all_Y.append(Y)


In [ ]:
print("total recordings:", len(all_X))
print("X mean (first):", all_X[0].mean())
print("X shape (first):", all_X[0].shape)
print("Y shape (first):", all_Y[0].shape)
print("unique labels (first):", np.unique(all_Y[0]))

print("X shape (first):", all_X[1].mean())
print("X shape (2nd):", all_X[1].shape)
print("Y shape (2nd):", all_Y[1].shape)
print("unique labels (2nd):", np.unique(all_Y[1]))

print("X shape (3rd):", all_X[2].shape)
print("Y shape (3rd):", all_Y[2].shape)
print("unique labels (3rd):", np.unique(all_Y[2]))